In [1]:
import pandas as pd
import googlemaps
from datetime import datetime
import requests
import time

In [ ]:
api_key = 'YOUR API KEY'

In [ ]:
def get_services(api_key, location, service_type, radius=5000):
    gmaps = googlemaps.Client(key=api_key)
    geocode_result = gmaps.geocode(location)
    if not geocode_result:
        return pd.DataFrame()  # Return an empty DataFrame if location is not found

    lat = geocode_result[0]["geometry"]["location"]["lat"]
    lng = geocode_result[0]["geometry"]["location"]["lng"]
    
    detailed_info = []

    def fetch_places(page_token=None):
        places_result = gmaps.places_nearby(location=(lat, lng), radius=radius, type=service_type, page_token=page_token)
        
        for place in places_result['results']:
            info = {
                'city': location.split(",")[0],  # Split to remove country for cleaner city names
                'name': place['name'],
                'formatted_address': place.get('vicinity', 'No address provided'),
                'place_id': place['place_id'],
                'latitude': place['geometry']['location']['lat'],
                'longitude': place['geometry']['location']['lng'],
                'rating': place.get('rating', 'No rating available'),
                'user_ratings_total': place.get('user_ratings_total', 0),
                'types': ', '.join(place.get('types', [])),
                'business_status': place.get('business_status', 'No business status available'),
                'permanently_closed': place.get('permanently_closed', False)
            }
            detailed_info.append(info)
        
        if 'next_page_token' in places_result:
            time.sleep(2)  # Wait for token to become valid
            fetch_places(places_result['next_page_token'])

    fetch_places()
    return pd.DataFrame(detailed_info)

# Select the cities (Cuneo Provincia)
test_cities = ["Arguello", "Barge", "Battifollo", "Bellino", "Bergolo",
    "Bonvicino", "Bosia", "Bossolasco", "Briga Alta", "Brossasco", "Camerana", "Canosio",
    "Caprauna", "Cartignano", "Casteldelfino", "Castelletto Uzzone", "Castelmagno",
    "Castelnuovo di Ceva", "Castiglione Tinella", "Castino", "Celle di Macra", "Cerretto Langhe",
    "Cissone", "Cortemilia", "Cossano Belbo", "Cravanzana", "Crissolo", "Demonte",
    "Elva", "Entracque", "Feisoglio", "Frassino", "Garessio", "Gorzegno", "Gottasecca",
    "Igliano", "Isasca", "Levice", "Limone Piemonte", "Macra", "Marmora", "Melle",
    "Mombarcaro", "Monesiglio", "Montà", "Montemale di Cuneo", "Montezemolo",
    "Murazzano", "Niella Belbo", "Oncino", "Ormea", "Ostana", "Paesana", "Paroldo",
    "Perletto", "Perlo", "Pezzolo Valle Uzzone", "Pietraporzio", "Pontechianale",
    "Pradleves", "Prazzo", "Priola", "Prunetto", "Rifreddo", "Roccabruna", "Rocchetta Belbo",
    "Saliceto", "Sambuco", "Sampeyre", "San Benedetto Belbo", "San Damiano Macra",
    "Sanfront", "Santo Stefano Belbo", "Serravalle Langhe", "Somano", "Stroppo",
    "Torre Bormida", "Torresina", "Vinadio", "Viola"]

# API Key and service type
service_type = 'establishment'

# Initialize a DataFrame to store all results
test_services = pd.DataFrame()

# Loop through each test city and gather services
for city in test_cities:
    print(f"Fetching data for {city}...")
    city_services = get_services(api_key, f"{city}, Italy", service_type)
    test_services = pd.concat([test_services, city_services], ignore_index=True)